In [3]:
# Wczytywanie pakietów 
import pandas as pd
import numpy as np

In [4]:
# Wczywytywanie ramek danych z plików gz.csv 
Posts = pd.read_csv("Posts.csv.gz")
Users = pd.read_csv("Users.csv.gz")
Badges = pd.read_csv("Badges.csv.gz")
Comments = pd.read_csv("Comments.csv.gz")
PostLinks = pd.read_csv("PostLinks.csv.gz")
Tags = pd.read_csv("Tags.csv.gz")
Votes = pd.read_csv("Votes.csv.gz")

In [5]:
##Tymczasowa baza danych do zapytan SQL
import tempfile, sqlite3
import os, os.path
baza = os.path.join(tempfile.mkdtemp(), "travel.db")
if os.path.isfile(baza):
    os.remove(baza)

conn = sqlite3.connect(baza)

Posts.to_sql("Posts", conn)
Users.to_sql("Users", conn)
Badges.to_sql("Badges", conn)
Comments.to_sql("Comments",conn)
PostLinks.to_sql("PostLinks", conn)
Tags.to_sql("Tags",conn)
Votes.to_sql("Votes", conn)


In [115]:
# zapytanie 1 sql 

zad1=pd.read_sql_query("""
SELECT
Users.DisplayName,
Users.Age,
Users.Location,
SUM(Posts.FavoriteCount) AS FavoriteTotal,
Posts.Title AS MostFavoriteQuestion,
MAX(Posts.FavoriteCount) AS MostFavoriteQuestionLikes
FROM Posts
JOIN Users ON Users.Id=Posts.OwnerUserId
WHERE Posts.PostTypeId=1
GROUP BY OwnerUserId
ORDER BY FavoriteTotal DESC
LIMIT 10
""",conn)

In [116]:
#Zapytanie 1 
# Robię inner joina ramek Posts i User po kolumnie OwnerUserId=Id
tmp = pd.merge(Posts,Users,left_on="OwnerUserId",right_on="Id")
# Wybór postów z TypeId == 1 
tmp = tmp.loc[tmp.PostTypeId==1]
# Wybór potrzebnych kolumn 
tmp = tmp[["DisplayName","Age","Location","FavoriteCount","Title","OwnerUserId"]]
# przygotowanie kolumn  FavoriteTotal i MostFavoriteQuestionLikes
tmpFO = tmp.groupby("OwnerUserId",sort=True).agg({"FavoriteCount":[np.max,np.sum]}).reset_index()
tmpFO.columns = tmpFO.columns.droplevel()
# łączenie z ramką tmp 
tmpFO.columns= ["OwnerUserId","MostFavoriteQuestionLikes","FavoriteTotal"]
tmp = pd.merge(tmp,tmpFO,left_on=["OwnerUserId","FavoriteCount"],right_on=["OwnerUserId","MostFavoriteQuestionLikes"],sort=True)
# nazywanie popranie kolumn 
tmp = tmp.rename(columns={"Title":"MostFavoriteQuestion"})
# wybieranie odpowiednich kolumn i sortownaie 
tmp = tmp.iloc[:,lambda tmp:[0,1,2,4,6,7]]
tmp.sort_values(by=['FavoriteTotal'], inplace=True,ascending=False)
tmp = tmp.iloc[:,lambda tmp:[0,1,2,5,3,4]]
# usuwanie NA niepotrzebne bo bierzemy pierwszych 10 
wynik = tmp.head(10).reset_index(drop=True)
# Sprawdzenie poprwaności 
wynik.equals(zad1)


True

In [117]:
#Zadanie 2  sql 

zad2=pd.read_sql_query("""
SELECT
Posts.ID,
Posts.Title,
Posts2.PositiveAnswerCount
FROM Posts
JOIN (
SELECT
Posts.ParentID,
COUNT(*) AS PositiveAnswerCount
FROM Posts
WHERE Posts.PostTypeID=2 AND Posts.Score>0
GROUP BY Posts.ParentID
) AS Posts2
ON Posts.ID=Posts2.ParentID
ORDER BY Posts2.PositiveAnswerCount DESC
LIMIT 10
""",conn)


,Id,Title,PositiveAnswerCount
0,250,Which European cities have bike rental station...,24
1,10,When traveling to a country with a different c...,20
2,13562,How do you choose a restaurant when travelling?,20
3,48775,How can I deal with people asking to switch se...,20
4,3220,Why would you wrap your luggage in plastic?,19
5,43660,Traveling in Europe Solo - 18 years old. Feasi...,19
6,30656,Long-life SIM cards in Europe,18
7,7663,Am I expected to tip wait staff in Europe?,17
8,59128,"Is there a way to prevent ""looking like a tour...",17
9,60446,Is it rude to ask if the food contains pork or...,17


In [118]:
# tworzę ramkę pomocniczą Posts2
Posts2 = Posts.query("PostTypeId==2 and Score>0")
# Grupuje i zliczam posty 
Posts2 = Posts2.groupby("ParentId",sort=True).size().reset_index()
# odpowiednio nazywam kolumny 
Posts2.columns = ["ParentId","PositiveAnswerCount"]

# przygotowuję ramkę Posts do joina 
tmp = Posts[["Id","Title"]]
# łączenie z ramką Posts2 
wynik = pd.merge(tmp,Posts2,left_on="Id",right_on="ParentId",sort=True)
# Sortownie po kolumnie PositiveAnswerCount i Id bo sql tak robi 
wynik.sort_values(by=['PositiveAnswerCount',"Id"], inplace=True,ascending=[False,True])
# wybieranie pierwszych 10  i własciwych kolumn 
wynik = wynik.head(10).reset_index(drop=True)
wynik = wynik.iloc[:,lambda tmp:[0,1,3]]
# sprawdzenie poprawności
wynik.equals(zad2)

True

In [6]:
# Zadanie 3 sql 
zad3=pd.read_sql_query("""
SELECT
Posts.Title,
UpVotesPerYear.Year,
MAX(UpVotesPerYear.Count) AS Count
FROM (
SELECT
PostId,
COUNT(*) AS Count,
STRFTIME('%Y', Votes.CreationDate) AS Year
FROM Votes
WHERE VoteTypeId=2
GROUP BY PostId, Year
) AS UpVotesPerYear
JOIN Posts ON Posts.Id=UpVotesPerYear.PostId
WHERE Posts.PostTypeId=1
GROUP BY Year
""",conn)


In [11]:
# tworzę ramkę pomocniczą 
UpVotesPerYear = Votes.query("VoteTypeId == 2")

# używma substringa do wybrania roku 
# czasem wyrzuca worning ale nie przeszkadaz on w działaniu
UpVotesPerYear.loc[:,"CreationDate"] = UpVotesPerYear.CreationDate.str.slice(0, 4)
# grupuje po Year i liczę Count 
UpVotesPerYear = UpVotesPerYear.groupby(["PostId","CreationDate"]).size().reset_index()
UpVotesPerYear.columns = ["PostId","Year","Count"]

# robię inner joina z ramką Posts najpierw wybierajć odpowienie kolumny oraz wiersze z PostTypeId == 1 
tmp = Posts.query("PostTypeId == 1 ")
tmp = tmp[["Title","Id"]]

wynik = pd.merge(tmp,UpVotesPerYear,left_on="Id",right_on="PostId")
# liczę max z kolumny Count 
wynik_tmp = wynik.groupby("Year",sort=True).agg({"Count":np.max}).reset_index()
# łączę ponownie powstałą ramkę z ramką wynikową tak aby zachować tytuły dla max(Count)
wynik = pd.merge(wynik,wynik_tmp,on=["Year","Count"])
# wybieram kolumny 
wynik = wynik.iloc[:,lambda tmp:[0,3,4]]
# sprawdzenie poprwaności
wynik.equals(zad3)

True

In [121]:
# Zadanie 4 sql 
zad4=pd.read_sql_query("""
SELECT
Questions.Id,
Questions.Title,
BestAnswers.MaxScore,
Posts.Score AS AcceptedScore,
BestAnswers.MaxScore-Posts.Score AS Difference
FROM (
SELECT Id, ParentId, MAX(Score) AS MaxScore
FROM Posts
WHERE PostTypeId==2
GROUP BY ParentId
) AS BestAnswers
JOIN (
SELECT * FROM Posts
WHERE PostTypeId==1
) AS Questions
ON Questions.Id=BestAnswers.ParentId
JOIN Posts ON Questions.AcceptedAnswerId=Posts.Id
WHERE Difference>50
ORDER BY Difference DESC
""",conn)


,Id,Title,MaxScore,AcceptedScore,Difference
0,99713,"In Germany, what are you supposed to do if you...",126,33,93
1,81376,What is way to eat rice with hands in front of...,120,30,90
2,76737,Why do many countries in the world still requi...,118,31,87
3,38177,How do you know if Americans genuinely/literal...,245,166,79
4,90636,Caught speeding 111 Mph (179 km/h) in Californ...,101,25,76
5,81492,Why is international first class much more exp...,90,21,69
6,94320,Strange looking region of France on Google Maps,71,7,64
7,80050,What's an easy way of making my luggage unique...,72,16,56


In [122]:
# tworzę ramkę BestAnswers 
BestAnswers = Posts.query(" PostTypeId ==2 ")
# liczę maksy grupując po PostId
BestAnswers_tmp = BestAnswers.groupby("ParentId").agg({"Score":np.max}).reset_index()
# łączę ponownie z ramką BestAnswers
BestAnswers = pd.merge(BestAnswers[["ParentId","Id","Score"]],BestAnswers_tmp,on=["ParentId","Score"])
# zmieniam nazwy kolumn 
BestAnswers = BestAnswers.rename(columns={"Score":"MaxScore"})
# Tworzę ramkę Question 
Question = Posts.query("PostTypeId==1 ")
# robię joina który ostatecznie tworzy ramkę Questions

Question= pd.merge(BestAnswers,Question,left_on="ParentId",right_on="Id")
# łoczę ramkę Questions z ramką Posts 
wynik = pd.merge(Posts,Question,right_on="AcceptedAnswerId",left_on="Id")
# wybieram odpowiendie kolumny 
wynik = wynik[["Id_y","Title_y","MaxScore","Score_x"]]
# nazwyam je poprawnie
wynik.columns = ["Id","Title","MaxScore","AcceptedScore"]
# tworzę kolumnę Difference
wynik["Difference"]=wynik["MaxScore"]-wynik["AcceptedScore"]
# Wybieram wiersze z Difference >50
wynik = wynik.query("Difference>50")
# sortuje po Difference malejąco 
wynik.sort_values(by=['Difference'], inplace=True,ascending=False)
wynik = wynik.reset_index(drop=True)
# Sprawdzam poprawność 
wynik.equals(zad4)

True

In [97]:
## zadanie 5 
zad5=pd.read_sql_query("""
SELECT
Posts.Title,
CmtTotScr.CommentsTotalScore
FROM (
SELECT
PostID,
UserID,
SUM(Score) AS CommentsTotalScore
FROM Comments
GROUP BY PostID, UserID
) AS CmtTotScr
JOIN Posts ON Posts.ID=CmtTotScr.PostID AND Posts.OwnerUserId=CmtTotScr.UserID
WHERE Posts.PostTypeId=1
ORDER BY CmtTotScr.CommentsTotalScore DESC
LIMIT 10
""",conn)


,Title,CommentsTotalScore
0,How to intentionally get denied entry to the U...,75
1,How can I deal with people asking to switch se...,32
2,What is France's traditional costume?,26
3,What's the longest scheduled public bus ride i...,25
4,Can I have a watermelon in hand luggage?,25
5,How does President Trump's travel ban affect n...,25
6,Caught speeding 111 Mph (179 km/h) in Californ...,24
7,Returning US Citizen lost passport in Canada,23
8,Legalities and safety concerns of visiting pro...,20
9,India just demonetized all Rs 500 & 1000 notes...,20


In [128]:
# Tworzę ramkę CmtTotScr 
CmtTotScr = Comments.groupby(["PostId","UserId"]).agg({"Score":np.sum}).reset_index()
# poprawiam nazwy kolumn 
CmtTotScr = CmtTotScr.rename(columns={"Score":"CommentsTotalScore"})

# Przygotowuję ramkę Posts wybierajać wiersze z PostTypeId==1 
tmp = Posts.query("PostTypeId == 1 ")
# łączę przygotowane ramki 
wynik = pd.merge(tmp,CmtTotScr,left_on=["Id","OwnerUserId"],right_on=["PostId","UserId"],sort=True)
# sortuje po CommentTotalScore i PostId
wynik.sort_values(by=["CommentsTotalScore","PostId"], inplace=True,ascending=[False,True])
# wybieram właściwe kolumny
wynik = wynik[["Title","CommentsTotalScore",]]
#wybieram 10 pierwszych wierszy 
wynik = wynik.head(10).reset_index(drop=True)
# sprawdzenie poprawności 
wynik.equals(zad5)


True

In [88]:
## zadanie 6
zad6=pd.read_sql_query("""
SELECT DISTINCT
Users.Id,
Users.DisplayName,
Users.Reputation,
Users.Age,
Users.Location
FROM (
SELECT
Name, UserID
FROM Badges
WHERE Name IN (
SELECT
Name
FROM Badges
WHERE Class=1
GROUP BY Name
HAVING COUNT(*) BETWEEN 2 AND 10
)
AND Class=1
) AS ValuableBadges
JOIN Users ON ValuableBadges.UserId=Users.Id
""",conn)


,Id,DisplayName,Reputation,Age,Location
0,108,Ankur Banerjee,31273,27.0,"London, UK"
1,19,VMAtm,18556,33.0,"Tampa, FL, United States"
2,101,Mark Mayo,121667,37.0,"Sydney, New South Wales, Australia"
3,466,iHaveacomputer,8360,NaN,Down underer
4,793,mindcorrosive,10531,32.0,Bulgaria
5,693,RoflcoptrException,33300,NaN,None
6,6669,Relaxed,69405,NaN,None
7,1737,Gayot Fow,70237,NaN,"London, United Kingdom"
8,39065,Pont,1004,NaN,Austria
9,19400,phoog,34342,50.0,"New York, NY"


In [124]:
# Towrzę ramkę pomocniczą
tmp = Badges.query("Class==1")
# zliczam grupując po kolumnie Name
tmp = tmp.groupby("Name").size().reset_index()
tmp = tmp.rename(columns={0:"Count"})
tmp =tmp.query("10>=Count>=2")
# Tworzę ramkę ValubleBadges
# tworzę listę z opserwcjami z Name 
tmp = tmp["Name"].tolist()
# Wybieram odpowiednie wierse z koluny Badges
ValubleBadges = Badges[Badges.Name.isin(tmp)]
ValubleBadges = ValubleBadges.query("Class==1")
# Teraz kolumny 
ValubleBadges = ValubleBadges[["Name","UserId"]]
# łączę z ramką Users
wynik = pd.merge(ValubleBadges,Users,left_on="UserId",right_on="Id")
wynik = wynik.iloc[:,lambda tmp:[7,5,10,3,9]].drop_duplicates().reset_index(drop=True)
# Sprawdzenie poprawności 
wynik.equals(zad6)

True

,index,Id,DisplayName,Reputation,Age,Location
0,0,108,Ankur Banerjee,31273,27.0,"London, UK"
1,2,19,VMAtm,18556,33.0,"Tampa, FL, United States"
2,3,101,Mark Mayo,121667,37.0,"Sydney, New South Wales, Australia"
3,5,466,iHaveacomputer,8360,NaN,Down underer
4,6,793,mindcorrosive,10531,32.0,Bulgaria
5,7,693,RoflcoptrException,33300,NaN,NaN
6,8,6669,Relaxed,69405,NaN,NaN
7,9,1737,Gayot Fow,70237,NaN,"London, United Kingdom"
8,10,39065,Pont,1004,NaN,Austria
9,11,19400,phoog,34342,50.0,"New York, NY"


In [125]:
## zadanie 7
zad7=pd.read_sql_query("""
SELECT
Posts.Title,
VotesByAge2.OldVotes
FROM Posts
JOIN (
SELECT
PostId,
MAX(CASE WHEN VoteDate = 'new' THEN Total ELSE 0 END) NewVotes,
MAX(CASE WHEN VoteDate = 'old' THEN Total ELSE 0 END) OldVotes,
SUM(Total) AS Votes
FROM (
SELECT
PostId,
CASE STRFTIME('%Y', CreationDate)
WHEN '2017' THEN 'new'
WHEN '2016' THEN 'new'
ELSE 'old'
END VoteDate,
COUNT(*) AS Total
FROM Votes
WHERE VoteTypeId=2
GROUP BY PostId, VoteDate
) AS VotesByAge
GROUP BY VotesByAge.PostId
HAVING NewVotes=0
) AS VotesByAge2 ON VotesByAge2.PostId=Posts.ID
WHERE Posts.PostTypeId=1
ORDER BY VotesByAge2.OldVotes DESC
LIMIT 10

""",conn)


,Title,OldVotes
0,Which European cities have bike rental station...,39
1,Why do hostels require you to 'rent' bedding?,28
2,What to do with your valuables on a low-cost h...,25
3,Can't check-in to a hotel because I am 18,25
4,What are some good ways to find things to expl...,24
5,Alarm Clock without Noise? To wake up in commo...,24
6,What times of the year are best for visiting F...,23
7,What is the business model of commercial free ...,23
8,Getting work on a cruise ship in order to travel,23
9,Carrying medicines internationally for a friend,23


In [126]:
# Tworzę ramkę VoteByAge
# wybieram obserwacje o VoteTypeId == 2
VoteByAge = Votes.query("VoteTypeId==2")
# Tworzę kolumnę Year
# czasem wyrzuca worning ale nie przeszkadaz on w działaniu
VoteByAge.loc[:,"CreationDate"] = VoteByAge.CreationDate.str.slice(0, 4)
# Zmieniam 2016/2017 na new pozostałe na old 
VoteByAge.loc[:,"CreationDate"] =np.where((VoteByAge["CreationDate"]=="2017") | (VoteByAge["CreationDate"]=="2016"),"new","old")
# Nazywam kolumnę poprawnie
VoteByAge = VoteByAge.rename(columns={"CreationDate":"Year"})
# zliczam ilość wierszy grupując po PostId ,Year
VoteByAge = VoteByAge.groupby(["PostId","Year"]).size().reset_index()
VoteByAge = VoteByAge.rename(columns={0:"Total"})
#Tworzę ramke VotesByAge2
# Tworzę kolumny New i Old Votes 
NewVotes= np.where(VoteByAge["Year"]=="new",VoteByAge["Total"],0)
OldVotes= np.where(VoteByAge["Year"]=="old",VoteByAge["Total"],0)
# dodaję kolumny NewVotes i OldVotes
VoteByAge["NewVotes"]= NewVotes
VoteByAge["OldVotes"]= OldVotes
# Liczę sumę Total i max po New i Old Votes
VoteByAge2 = VoteByAge.groupby("PostId",sort=True).agg({"NewVotes":np.max,"OldVotes":np.max,"Total":np.sum}).reset_index()
VoteByAge2 = VoteByAge2.query("NewVotes==0")
# Przygotwanie ramki Posts do joina z VotesByAge2
tmp = Posts.query("PostTypeId==1")

# łączę przygotowne ramki
wynik = pd.merge(tmp,VoteByAge2,left_on="Id",right_on="PostId")
#sortuję po mergu
wynik.sort_values(by=["OldVotes","PostId"], inplace=True,ascending=[False,True],na_position="last")
# wybieram potrzebne kolumny 
wynik = wynik[["Title","OldVotes"]]
# wybieram pierwsze 10 
wynik=wynik.head(10).reset_index(drop=True)
# sprawdenie poprwaności 
wynik.equals(zad7)

True

,Title,OldVotes
0,Which European cities have bike rental station...,39
1,Why do hostels require you to 'rent' bedding?,28
2,What to do with your valuables on a low-cost h...,25
3,Can't check-in to a hotel because I am 18,25
4,What are some good ways to find things to expl...,24
5,Alarm Clock without Noise? To wake up in commo...,24
6,What times of the year are best for visiting F...,23
7,What is the business model of commercial free ...,23
8,Getting work on a cruise ship in order to travel,23
9,Carrying medicines internationally for a friend,23
